### lab-4 tree / Классификация. Деревья решений

Мисюрина Ирина

1. Считать в `pandas.DataFrame` любой источник данных: CSV, JSON, Excel-файл, HTML-таблицу и т.п.

In [1]:
!wget https://raw.githubusercontent.com/IraMeis/ai-tech-spring24/main/datasets/water_potability.csv

--2024-05-13 12:48:04--  https://raw.githubusercontent.com/IraMeis/ai-tech-spring24/main/datasets/water_potability.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 521910 (510K) [text/plain]
Saving to: ‘water_potability.csv’

water_potability.cs 100%[===================>] 509.68K  --.-KB/s    in 0.04s   

2024-05-13 12:48:04 (12.0 MB/s) - ‘water_potability.csv’ saved [521910/521910]



In [2]:
import pandas as pd
df = pd.read_csv('water_potability.csv')
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


2. Датасет и подготовка данных:
   - Привести описание датасета.

|Название признака|Значение признака|Единица измерения|
|-----------------|-----------------|-----------------|
|ph|ph - float|б/м|
|hardness|жесткость - float|б/м|
|solids|твердые вещества - float|мг/л|
|chloramines|хлорамины - float|мг/л|
|sulfate|сульфаты - float|мг/л|
|conductivity|проводимость - float|мкС/см|
|organic carbon|органический углерод - float|мг/л|
|trihalomethanes|тригалометаны - float|мг/л|
|turbidity|мутность - float|NTU|
|potability|пригодность для питья|0 - непригодна, 1 - пригодна|

   - Осуществить предобработку данных (избавиться от `null`, убрать некоторые признаки и т.п.) - "подчистить данные".
   - Закодировать категориальные признаки при необходимости.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB


In [4]:
df.fillna(df.mean(), inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               3276 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          3276 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3276 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB


   - Нормализовать данные.

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
X = df.drop(['Potability'], axis=1)
y = df['Potability']

In [8]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,0.505771,0.571139,0.336096,0.543891,0.680385,0.669439,0.313402,0.699753,0.286091
1,0.265434,0.297400,0.300611,0.491839,0.581699,0.719411,0.497319,0.450999,0.576793
2,0.578509,0.641311,0.321619,0.698543,0.581699,0.414652,0.562017,0.532866,0.303637
3,0.594055,0.605536,0.356244,0.603314,0.647347,0.317880,0.622089,0.808065,0.601015
4,0.649445,0.484851,0.289922,0.484900,0.514545,0.379337,0.358555,0.253606,0.496327


   - Разбить выборку на обучающую и тестовую.
     > Далее используем обучающую выборку, в том числе для метрик.

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 42)
print(y_train.shape, y_test.shape)

(2293,) (983,)


3. Дерево решений:
   - С использованием `GridSearchCV` осуществить подбор гиперпараметра `DecisionTreeClassifier` (как минимум `max_depth`, `max_features`, другие параметры - по желанию.)

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [35]:
grid = {
    'max_depth': range(1, 15),
    'max_features': range(2, 21, 2)
}

In [36]:
model = DecisionTreeClassifier()
search = GridSearchCV(model, grid, cv = 5)
search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 15),
                         'max_features': range(2, 21, 2)})

- Вывести значения гиперпараметра и метрик для наилучшей модели `DecisionTreeClassifier` ($accuracy$, $precision$, $recall$, $\textit{f-measure}$).

In [37]:
search.best_params_

{'max_depth': 7, 'max_features': 6}

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [39]:
best_model = search.best_estimator_
pred = best_model.predict(X_train)
print(f'accuracy: {accuracy_score(y_train, pred)}')
print(f'precision: {precision_score(y_train, pred)}')
print(f'recall: {recall_score(y_train, pred)}')
print(f'f-measure: {f1_score(y_train, pred)}')

accuracy: 0.7108591365023986
precision: 0.825065274151436
recall: 0.34649122807017546
f-measure: 0.48803088803088807


   - Для полученного наилучшего дерева вывести `feature_importances`, отсортировать их по убыванию.

In [40]:
features = zip(X_train.columns, best_model.feature_importances_)
features = sorted(features, key=lambda x: x[1], reverse=True)
for k, v in features:
    print(f"{k} {v:.4f}")

Chloramines 0.1898
ph 0.1877
Sulfate 0.1865
Hardness 0.1288
Solids 0.0842
Turbidity 0.0757
Conductivity 0.0582
Trihalomethanes 0.0477
Organic_carbon 0.0415


   - Осуществить фильтрацию признаков (по какому-нибудь значению порога важности признака).

In [41]:
import numpy as np
f_filtered = [k for k, v in features if v > np.median([v for _, v in features])]
f_filtered

['Chloramines', 'ph', 'Sulfate', 'Hardness']

   - Подобрать лучшую модель с использованием `GridSearchCV` на обучающей выборке с отфильтрованными признаками.

In [42]:
X_train_filtered = X_train[f_filtered]
X_train_filtered.head()

,Chloramines,ph,Sulfate,Hardness
2985,0.601828,0.505771,0.718714,0.511489
1073,0.464286,0.514531,0.405908,0.438944
3140,0.430672,0.505771,0.600370,0.708746
2643,0.716658,0.432630,0.581699,0.596078
2174,0.415924,0.684371,0.581699,0.637901


In [43]:
search_f = GridSearchCV(model, grid, cv = 5)
search_f.fit(X_train_filtered, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 15),
                         'max_features': range(2, 21, 2)})

   - Вывести полученные гиперпараметры лучшей модели.

In [44]:
search_f.best_params_

{'max_depth': 8, 'max_features': 14}

   - Сравнить метрики до и после фильтрации признаков лучших моделей.

In [45]:
best_model_f = search_f.best_estimator_

In [46]:
print('no filter\n')
pred = best_model.predict(X_train)
print(f'accuracy: {accuracy_score(y_train, pred)}')
print(f'precision: {precision_score(y_train, pred)}')
print(f'recall: {recall_score(y_train, pred)}')
print(f'f-measure: {f1_score(y_train, pred)}')
print('\nfilter\n')
pred = best_model_f.predict(X_train_filtered)
print(f'accuracy: {accuracy_score(y_train, pred)}')
print(f'precision: {precision_score(y_train, pred)}')
print(f'recall: {recall_score(y_train, pred)}')
print(f'f-measure: {f1_score(y_train, pred)}')

no filter

accuracy: 0.7108591365023986
precision: 0.825065274151436
recall: 0.34649122807017546
f-measure: 0.48803088803088807

filter

accuracy: 0.7108591365023986
precision: 0.7902097902097902
recall: 0.3717105263157895
f-measure: 0.5055928411633109


accuracy не поменялась, соотношение precision-recall улучшилось

4. Случайный лес
   - Построить случайный лес (`RandomForestClassifier`), c использованием `GridSearchCV` осуществить подбор гиперпараметра.

In [47]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier()

In [48]:
forest_grid = {
    'max_depth': range(1, 15),
    'n_estimators': range(10, 201, 10)
}

In [49]:
forest_search = GridSearchCV(forest_model, forest_grid, cv = 5)
forest_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 15),
                         'n_estimators': range(10, 201, 10)})

   - Вывести полученные гиперпараметры лучшей модели случайного леса.

In [50]:
forest_search.best_params_

{'max_depth': 14, 'n_estimators': 150}

   - Осуществить фильтрацию признаков.

In [51]:
forest_best_model = forest_search.best_estimator_
forest_features = zip(X_train.columns, forest_best_model.feature_importances_)
forest_features = sorted(forest_features, key=lambda x: x[1], reverse=True)
for k, v in forest_features:
    print(f"{k} {v:.4f}")

Sulfate 0.1358
ph 0.1286
Hardness 0.1242
Chloramines 0.1208
Solids 0.1163
Organic_carbon 0.0978
Conductivity 0.0964
Turbidity 0.0934
Trihalomethanes 0.0869


In [52]:
forest_f_filtered = [k for k, v in forest_features if v > np.median([v for _, v in forest_features])]
forest_f_filtered

['Sulfate', 'ph', 'Hardness', 'Chloramines']

   - Подобрать лучшую модель с использованием `GridSearchCV` на обучающей выборке с отфильтрованными признаками.

In [53]:
X_train_forest_filtered = X_train[forest_f_filtered]
X_train_forest_filtered.head()

,Sulfate,ph,Hardness,Chloramines
2985,0.718714,0.505771,0.511489,0.601828
1073,0.405908,0.514531,0.438944,0.464286
3140,0.600370,0.505771,0.708746,0.430672
2643,0.581699,0.432630,0.596078,0.716658
2174,0.581699,0.684371,0.637901,0.415924


In [54]:
forest_search_f = GridSearchCV(forest_model, forest_grid, cv = 5)
forest_search_f.fit(X_train_forest_filtered, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 15),
                         'n_estimators': range(10, 201, 10)})

   - Вывести полученные гиперпараметры лучшей модели случайного леса.

In [55]:
forest_search_f.best_params_

{'max_depth': 13, 'n_estimators': 120}

   - Сравнить метрики до и после фильтрации признаков лучших моделей.

In [56]:
forest_best_model_f = forest_search_f.best_estimator_

In [57]:
print('no filter\n')
pred =  forest_best_model.predict(X_train)
print(f'accuracy: {accuracy_score(y_train, pred)}')
print(f'precision: {precision_score(y_train, pred)}')
print(f'recall: {recall_score(y_train, pred)}')
print(f'f-measure: {f1_score(y_train, pred)}')
print('\nfilter\n')
pred = forest_best_model_f.predict(X_train_forest_filtered)
print(f'accuracy: {accuracy_score(y_train, pred)}')
print(f'precision: {precision_score(y_train, pred)}')
print(f'recall: {recall_score(y_train, pred)}')
print(f'f-measure: {f1_score(y_train, pred)}')

no filter

accuracy: 0.9415612734409071
precision: 1.0
recall: 0.8530701754385965
f-measure: 0.9207100591715977

filter

accuracy: 0.8796336676842564
precision: 0.9984326018808778
recall: 0.6984649122807017
f-measure: 0.8219354838709678


Метрики немного ухудшились

5. Метод ближайших соседей:
   - С использованием `GridSearchCV` осуществить подбор гиперпараметра `KNeighborsClassifier` (`n_neighbors`).

In [58]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()

In [59]:
knn_grid = {
    'n_neighbors': range(2, 21),
}

In [60]:
knn_search = GridSearchCV(knn_model, knn_grid, cv = 5)
knn_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(2, 21)})

   - Вывести значения гиперпараметра и метрик для наилучшей модели.

In [61]:
knn_search.best_params_

{'n_neighbors': 8}

   - Осуществить фильтрацию признаков.

Для KNN feature_importances_ нет, поэтому возьмем фильтр для DecisionTree

In [62]:
knn_filtered = f_filtered
X_train_knn_filtered = X_train[knn_filtered]
X_train_knn_filtered.head()

,Chloramines,ph,Sulfate,Hardness
2985,0.601828,0.505771,0.718714,0.511489
1073,0.464286,0.514531,0.405908,0.438944
3140,0.430672,0.505771,0.600370,0.708746
2643,0.716658,0.432630,0.581699,0.596078
2174,0.415924,0.684371,0.581699,0.637901


   - Подобрать лучшую модель с использованием `GridSearchCV` на обучающей выборке с отфильтрованными признаками.

In [63]:
knn_search_f = GridSearchCV(knn_model, knn_grid, cv = 5)
knn_search_f.fit(X_train_knn_filtered, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(2, 21)})

   - Вывести полученные гиперпараметры лучшей модели случайного леса.

In [64]:
knn_search_f.best_params_

{'n_neighbors': 20}

   - Сравнить метрики до и после фильтрации признаков.

In [65]:
knn_best_model = knn_search.best_estimator_
knn_best_model_f = knn_search_f.best_estimator_

In [66]:
print('no filter\n')
pred = knn_best_model.predict(X_train)
print(f'accuracy: {accuracy_score(y_train, pred)}')
print(f'precision: {precision_score(y_train, pred)}')
print(f'recall: {recall_score(y_train, pred)}')
print(f'f-measure: {f1_score(y_train, pred)}')
print('\nfilter\n')
pred = knn_best_model_f.predict(X_train_knn_filtered)
print(f'accuracy: {accuracy_score(y_train, pred)}')
print(f'precision: {precision_score(y_train, pred)}')
print(f'recall: {recall_score(y_train, pred)}')
print(f'f-measure: {f1_score(y_train, pred)}')

no filter

accuracy: 0.7069341474051462
precision: 0.7985074626865671
recall: 0.3519736842105263
f-measure: 0.4885844748858447

filter

accuracy: 0.6942869603139992
precision: 0.7308533916849015
recall: 0.36622807017543857
f-measure: 0.48794740686632576


6. Если наблюдается улучшение метрик после фильтрации признаков хотя бы для одной из моделей, то для набора отфильтрованных признаков (пересечение множеств отфильтрованных признаков каждой модели или объединение множеств &ndash; не особо важно, главное описать, каким образом получен новый subset данных) заново построить наилучшие модели `KNeighborsClassifier`, `DecisionTreeClassifier`, `RandomForestClassifier`, сравнить модели в пункте 7 на одинаковом полученном наборе отфильтрованных признаков. Иначе &ndash; пропустить этот пункт.

У DecisionTree метрики несколько улучшились. Далее берем объединение всех отфильтровнных признаков - ['Sulfate', 'ph', 'Chloramines', 'Solids', 'Hardness']

In [67]:
filter_features = ['Sulfate', 'ph', 'Chloramines', 'Solids', 'Hardness']

tree_search = GridSearchCV(model, grid, cv = 5)
tree_search.fit(X_train[filter_features], y_train)
forest_search = GridSearchCV(forest_model, forest_grid, cv = 5)
forest_search.fit(X_train[filter_features], y_train)
knn_search = GridSearchCV(knn_model, knn_grid, cv = 5)
knn_search.fit(X_train[filter_features], y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(2, 21)})

In [68]:
best_tree = tree_search.best_estimator_
best_forest = forest_search.best_estimator_
best_knn = knn_search.best_estimator_

7. Оценка качества построенных моделей:
   - Визуализировать любое полученное дерево решений.
     > Для вывода названий признаков в граф необходимо задать значение аргумента `feature_names` в `sklearn.tree.export_graphviz`, для вывода названий классов &ndash; `class_names` (перед кодированием целевого признака можно сохранить названия в отдельный массив).

In [79]:
from sklearn.tree import export_graphviz

export_graphviz(
    best_tree,
    feature_names=filter_features,
    class_names=['bad','ok'],
    out_file = 'tree.dot',
    filled = True
)

In [80]:
!dot -Tpng ./tree.dot -o ./tree.png

если картинка не отображается, визуализация дерева дублируется в /utils

   - Сравнить лучшие модели `KNeighborsClassifier`, `DecisionTreeClassifier`, `RandomForestClassifier` на **тестовой выборке**. Привести значения метрик $accuracy$, $precision$, $recall$, $\textit{f-measure}$.

In [81]:
print('decision tree\n')
pred = best_tree.predict(X_test[filter_features])
print(f'accuracy: {accuracy_score(y_test, pred)}')
print(f'precision: {precision_score(y_test, pred)}')
print(f'recall: {recall_score(y_test, pred)}')
print(f'f-measure: {f1_score(y_test, pred)}')
print('\nrandom forest\n')
pred = best_forest.predict(X_test[filter_features])
print(f'accuracy: {accuracy_score(y_test, pred)}')
print(f'precision: {precision_score(y_test, pred)}')
print(f'recall: {recall_score(y_test, pred)}')
print(f'f-measure: {f1_score(y_test, pred)}')
print('\nknn\n')
pred = best_knn.predict(X_test[filter_features])
print(f'accuracy: {accuracy_score(y_test, pred)}')
print(f'precision: {precision_score(y_test, pred)}')
print(f'recall: {recall_score(y_test, pred)}')
print(f'f-measure: {f1_score(y_test, pred)}')

decision tree

accuracy: 0.6449643947100712
precision: 0.5562913907284768
recall: 0.22950819672131148
f-measure: 0.32495164410058025

random forest

accuracy: 0.6948118006103764
precision: 0.6833333333333333
recall: 0.3360655737704918
f-measure: 0.4505494505494505

knn

accuracy: 0.7080366225839267
precision: 0.7206703910614525
recall: 0.3524590163934426
f-measure: 0.47339449541284395


KNN показал более приемлемые метрики по сравнению с деревом решений и случайным лесом